In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import tensorflow as tf
import jieba
import pandas as pd
import gensim
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.test.is_gpu_available())
from sklearn.model_selection import train_test_split

1.14.0
False


In [7]:
comments = pd.read_csv('../../datasource/movie_comments.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [198]:
def stopword(sentence):
    with open("../../datasource/chinese_stopwords.txt", 'r') as f:
        stopwords = [line.replace('\n', '') for line in f.readlines() if line != '\n']
    return [sen for sen in sentence if sen not in stopwords and sen not in ['\r\n', '\r']]       

In [212]:
data, target = [], []
def reformat():
    for i, comm in enumerate(comments.values):
        try:
            comm_re = ''.join(re.findall(r'[\S]', comm[3]))
            cm = stopword(jieba.lcut(comm_re))
            star = int(comm[4])
            if len(cm) > 3 and star:
#                 print(cm)
                data.append(' '.join(cm))
                target.append(1 if star > 2 else 0)            
        except Exception as e:
            print(e)
            continue
        if i%100000 == 0:
            print(i, comm)

In [213]:
reformat()

0 ['1' 'https://movie.douban.com/subject/26363254/' '战狼2'
 '吴京意淫到了脑残的地步，看了恶心想吐' '1']
invalid literal for int() with base 10: 'star'
100000 ['98655' 'https://movie.douban.com/subject/26088403/' '检察官外传 검사외전'
 '善良地活着吧！\n捍卫自己在意的事物！' '4']
200000 [198655 'https://movie.douban.com/subject/25839662/' '45周年 45 Years'
 '#2016十佳候选#“就怕鬼一样的前女友”。被发现的不仅仅是逝去的爱人，还是这个已经年老的欧洲昨日的（充满可能性却错失了的）六十年代。看上去就是个老年中产阶级情节剧，不过低下政治暗流涌动——没有什么比“胎死腹中”更加说明问题的了。'
 5]


In [214]:
sentences = [sentence.split(' ') for sentence in data]

In [180]:
# sentences = gensim.models.word2vec.LineSentence(source=da)

In [217]:
model = gensim.models.Word2Vec(sentences=sentences, min_count=1)

In [335]:
model.save('word2vec.model')

In [218]:
# 取所有词平均作为句子向量
sentence_embedding = []
for i, sentence in enumerate(sentences):
    sent_vec = np.zeros((100, ))
    for sen in sentence:
        try:
            sent_vec += model[sen]
        except Exception as e:
            print(e)
            continue
    if np.sum(sent_vec) != 0.:
        sent_vec = sent_vec / len(sentence)
        sentence_embedding.append(sent_vec)
    else:
        print('delete target: ', i)
        target.pop(i)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [322]:
sentence_embedding_bak = np.r_[sentence_embedding]
train_x, test_x, train_y, test_y = train_test_split(sentence_embedding_bak, target, train_size=0.75)

In [454]:
class_model = keras.Sequential()
class_model.add(keras.layers.Dense(16, activation='relu', input_shape=(100,)))
class_model.add(keras.layers.Dense(1, activation='sigmoid'))
class_model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 16)                1616      
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 17        
Total params: 1,633
Trainable params: 1,633
Non-trainable params: 0
_________________________________________________________________


In [455]:
class_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'] )

In [456]:
history = class_model.fit(test_x, 
                    test_y, 
                    epochs=40, 
                    batch_size=512, 
                    validation_data=(train_x, train_y), 
                    verbose=1)

Train on 55641 samples, validate on 129826 samples
Epoch 1/40
55641/55641 [==============================] - 1s 13us/sample - loss: 0.4850 - acc: 0.7934 - val_loss: 0.4342 - val_acc: 0.8074
Epoch 2/40
55641/55641 [==============================] - 0s 6us/sample - loss: 0.4215 - acc: 0.8141 - val_loss: 0.4134 - val_acc: 0.8187
Epoch 3/40
55641/55641 [==============================] - 0s 6us/sample - loss: 0.4118 - acc: 0.8204 - val_loss: 0.4090 - val_acc: 0.8209
Epoch 4/40
55641/55641 [==============================] - 0s 6us/sample - loss: 0.4083 - acc: 0.8231 - val_loss: 0.4061 - val_acc: 0.8228
Epoch 5/40
55641/55641 [==============================] - 0s 7us/sample - loss: 0.4053 - acc: 0.8257 - val_loss: 0.4040 - val_acc: 0.8235
Epoch 6/40
55641/55641 [==============================] - 0s 6us/sample - loss: 0.4033 - acc: 0.8259 - val_loss: 0.4021 - val_acc: 0.8245
Epoch 7/40
55641/55641 [==============================] - 0s 6us/sample - loss: 0.4013 - acc: 0.8267 - val_loss: 0.4006 

# xgboost

In [342]:
import xgboost

In [343]:
model = xgboost.XGBClassifier()

In [350]:
model.fit(train_x[: 10000], train_y[:10000])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [346]:
from sklearn.metrics import accuracy_score

In [351]:
y_hat = model.predict(test_x[: 100])

In [352]:
accuracy_score(test_y[: 100], y_hat)

0.82

In [349]:
help(model)

Help on XGBClassifier in module xgboost.sklearn object:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, silent=None, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, **kwargs)
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  Parameters
 |  ----------
 |  max_depth : int
 |      Maximum tree depth for base learners.
 |  learning_rate : float
 |      Boosting learning rate (xgb's "eta")
 |  n_estimators : int
 |      Number of trees to fit.
 |  verbosity : int
 |      The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 |  silent : boolean
 |      Whether to print messages while running boosting. Depreca

In [383]:
train_data.shape

(25000, 256)

In [390]:
sentence_embedding_bak_ = sentence_embedding_bak 

In [391]:
sentence_embedding_bak_ = keras.preprocessing.sequence.pad_sequences(sentence_embedding_bak_,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)


In [393]:
sentence_embedding_bak_.shape

(185467, 256)

In [402]:
train_x, test_x = sentence_embedding_bak_[: 10000], sentence_embedding_bak_[10000: 25000]

In [405]:
train_y, test_y = np.array(target)[: 10000], np.array(target)[10000: 25000]

In [449]:
train_x, test_x, train_y, test_y = train_test_split(sentence_embedding_bak, np.array(target), test_size=0.3, random_state=7)

In [457]:
class_model.predict(train_x[:1,:])

array([[0.908958]], dtype=float32)